# Transfer Learning

In [1]:
import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear inline in the
# notebook rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
# import the necessary packages
from keras.applications import VGG16
import argparse

# whether or not to include top of CNN
include_top = -1

# load the VGG16 network
print("[INFO] loading network...")
model = VGG16(weights="imagenet", include_top=include_top>0)
print("[INFO] showing layers...")

# loop over the layers in the network and display them to the console
for (i, layer) in enumerate(model.layers):
    print("[INFO] {}\t{}".format(i, layer.__class__.__name__))

Using TensorFlow backend.
/home/pedro/anaconda3/envs/computer-vision/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pedro/anaconda3/envs/computer-vision/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pedro/anaconda3/envs/computer-vision/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

[INFO] loading network...
[INFO] showing layers...
[INFO] 0	InputLayer
[INFO] 1	Conv2D
[INFO] 2	Conv2D
[INFO] 3	MaxPooling2D
[INFO] 4	Conv2D
[INFO] 5	Conv2D
[INFO] 6	MaxPooling2D
[INFO] 7	Conv2D
[INFO] 8	Conv2D
[INFO] 9	Conv2D
[INFO] 10	MaxPooling2D
[INFO] 11	Conv2D
[INFO] 12	Conv2D
[INFO] 13	Conv2D
[INFO] 14	MaxPooling2D
[INFO] 15	Conv2D
[INFO] 16	Conv2D
[INFO] 17	Conv2D
[INFO] 18	MaxPooling2D


In [3]:
# import the necessary packages
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense

In [4]:
class FCHeadNet:
    
    def build(self, baseModel, classes, D):
        # initialize the head model that will be placed on top of
        # the base, then add a FC layer
        headModel = baseModel.output
        headModel = Flatten(name="flatten")(headModel)
        headModel = Dense(D, activation="relu")(headModel)
        headModel = Dropout(0.5)(headModel)

        # add a softmax layer
        headModel = Dense(classes, activation="softmax")(headModel)

        # return the model
        return headModel

# Fine Tuning

In [5]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from utils.image_preprocessor import ImagePreprocessor
from utils.image_preprocessor import AspectAwarePreprocessor
from utils.image_preprocessor import ImageToArrayPreprocessor

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.optimizers import SGD
from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model

import numpy as np
import os

In [6]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

In [7]:
from keras.datasets import cifar10
((trainX, trainY), (testX, testY)) = cifar10.load_data()
labelNames = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [8]:
train_sample_num = 1000
test_sample_num = 200

In [9]:
trainX = trainX[:train_sample_num]
trainY = trainY[:train_sample_num]
print(trainX.shape)
print(trainY.shape)

(1000, 32, 32, 3)
(1000, 1)


In [10]:
testX = testX[:test_sample_num]
testY = testY[:test_sample_num]
print(testX.shape)
print(testY.shape)

(200, 32, 32, 3)
(200, 1)


In [11]:
aap = AspectAwarePreprocessor(224, 224)
iap = ImageToArrayPreprocessor()
ip = ImagePreprocessor(preprocessors=[aap, iap])

In [12]:
trainX = ip.preprocess(trainX)
testX = ip.preprocess(testX)

In [13]:
trainX = trainX / 255.0
testX = testX / 255.0

In [14]:
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [15]:
print(trainX.shape)
print(trainY.shape)

(1000, 224, 224, 3)
(1000, 10)


In [16]:
print(testX.shape)
print(testY.shape)

(200, 224, 224, 3)
(200, 10)


In [ ]:
# path to output model
model = 

In [20]:
# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# initialize the new head of the network, a set of FC layers
# followed by a softmax classifier
headModel = FCHeadNet().build(baseModel, len(labelNames), 256)

# place the head FC model on top of the base model -- this will
# become the actual model we will train
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they
# will *not* be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False

# compile our model (this needs to be done after our setting our
# layers to being non-trainable
print("[INFO] compiling model...")

opt = RMSprop(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the head of the network for a few epochs (all other
# layers are frozen) -- this will allow the new FC layers to
# start to become initialized with actual "learned" values
# versus pure random
print("[INFO] training head...")
model.fit_generator(aug.flow(trainX, trainY, batch_size=32), 
                    validation_data=(testX, testY), epochs=25, 
                    steps_per_epoch=len(trainX) // 32, verbose=1)

# evaluate the network after initialization
print("[INFO] evaluating after initialization...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames))

[INFO] compiling model...
[INFO] training head...
Epoch 1/25
31/31 [==============================] - 8s 263ms/step - loss: 4.3442 - acc: 0.1270 - val_loss: 2.2730 - val_acc: 0.2000
Epoch 2/25
31/31 [==============================] - 7s 221ms/step - loss: 2.2723 - acc: 0.1692 - val_loss: 2.2326 - val_acc: 0.1500
Epoch 3/25
31/31 [==============================] - 7s 222ms/step - loss: 2.2250 - acc: 0.1410 - val_loss: 2.2035 - val_acc: 0.2100
Epoch 4/25
31/31 [==============================] - 7s 227ms/step - loss: 2.2204 - acc: 0.1643 - val_loss: 2.2676 - val_acc: 0.1900
Epoch 5/25
31/31 [==============================] - 7s 225ms/step - loss: 2.1276 - acc: 0.2017 - val_loss: 2.1751 - val_acc: 0.2250
Epoch 6/25
31/31 [==============================] - 7s 231ms/step - loss: 2.1311 - acc: 0.2078 - val_loss: 2.0260 - val_acc: 0.2450
Epoch 7/25
31/31 [==============================] - 7s 228ms/step - loss: 2.1026 - acc: 0.1885 - val_loss: 2.0816 - val_acc: 0.2500
Epoch 8/25
31/31 [========

/home/pedro/anaconda3/envs/computer-vision/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable
for layer in baseModel.layers[15:]:
    layer.trainable = True

# for the changes to the model to take affect we need to recompile
# the model, this time using SGD with a *very* small learning rate
print("[INFO] re-compiling model...")
opt = SGD(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers
print("[INFO] fine-tuning model...")
model.fit_generator(aug.flow(trainX, trainY, batch_size=32), 
                    validation_data=(testX, testY), epochs=100, 
                    steps_per_epoch=len(trainX) // 32, verbose=1)

# evaluate the network on the fine-tuned model
print("[INFO] evaluating after fine-tuning...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames))

# save the model to disk
print("[INFO] serializing model...")
model.save(args["model"])